In [211]:
import pandas as pd

In [212]:
df = pd.read_csv('train.csv',encoding='cp949')

In [213]:
df.head(2)

,차변계정,일자,코드,거래처,품명,공급가액,세액,국세청,추천유형,전표유형,전표유형.1,Unnamed: 11,대변계정,분개전송,전표상태,사업자등록번호,업태,업종
0,(판)지급수수료,01-09,134.0,（주）고센인코퍼,NaN,37273,3727.0,불공제,불공제,매입,카과,NaN,현금,○,전송완료,120-81-33347,NaN,NaN
1,(판)소모품비,01-16,298.0,(주) 제이포유컴퍼니,NaN,321819,32181.0,불공제,불공제,매입,카과,NaN,현금,○,전송완료,513-86-01926,NaN,NaN


In [214]:
df.columns

Index(['차변계정', '일자', '코드', '거래처', '품명', '공급가액', '세액', '국세청', '추천유형', '전표유형',
       '전표유형.1', 'Unnamed: 11', '대변계정', '분개전송', '전표상태', '사업자등록번호', '업태', '업종'],
      dtype='object')

In [215]:
cols = ['차변계정', '공급가액','세액','사업자등록번호']  # 거래처는 사업자등록번호가 대처

In [216]:
df2 = df[cols]
df2.head(2)

,차변계정,공급가액,세액,사업자등록번호
0,(판)지급수수료,37273,3727.0,120-81-33347
1,(판)소모품비,321819,32181.0,513-86-01926


평일 공휴일 주말 여부 추가

In [217]:
# 현재년도 추가
year = 2024
df2.loc[:,'날자'] =   pd.to_datetime(f"{year}-"+df['일자'],format="%Y-%m-%d")

<ipython-input-217-cb3860a2d812>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[:,'날자'] =   pd.to_datetime(f"{year}-"+df['일자'],format="%Y-%m-%d")


In [218]:
# 데이터중에 na가 있는 row는 모두 출력 해서 확인
df2[df2.isna().any(axis=1)]

,차변계정,공급가액,세액,사업자등록번호,날자
45,(판)접대비(기업업무추진비),50000,NaN,106-59-00136,2024-06-12
60,(판)여비교통비,3000,NaN,358-87-00860,2024-01-03
65,(판)접대비(기업업무추진비),90000,NaN,211-09-47307,2024-01-05
88,(판)지급수수료,45000,NaN,777-33-00694,2024-01-26
93,(판)소모품비,990000,NaN,113-86-10017,2024-01-29
...,...,...,...,...,...
3703,(판)여비교통비,32300,NaN,129-86-38970,2024-06-23
3709,미추천,86800,NaN,664-93-00733,2024-06-24
3733,(판)여비교통비,3000,NaN,104-81-83559,2024-06-28
3739,(판)지급수수료,66000,NaN,606-71-00375,2024-06-29


In [219]:
# 세액컬럼의 na 를 0으로 채운다
df2['세액'] = df2['세액'].fillna(0)

<ipython-input-219-0d004a52064e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['세액'] = df2['세액'].fillna(0)


In [220]:
df2.isna().sum()

,0
차변계정,0
공급가액,0
세액,0
사업자등록번호,0
날자,0


In [221]:
# 주말 여부 확인
df2['요일'] = df2['날자'].dt.day_name()
df2['주말여부'] = df2['날자'].dt.weekday >=5  # 5,6 토요일 일요일

<ipython-input-221-30eb3bc2470c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['요일'] = df2['날자'].dt.day_name()
<ipython-input-221-30eb3bc2470c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['주말여부'] = df2['날자'].dt.weekday >=5  # 5,6 토요일 일요일


In [222]:
df2.tail()

,차변계정,공급가액,세액,사업자등록번호,날자,요일,주말여부
3736,(판)여비교통비,3636,364.0,805-33-00999,2024-06-28,Friday,False
3737,(판)여비교통비,2909,291.0,439-26-01167,2024-06-29,Saturday,True
3738,(판)여비교통비,8182,818.0,584-08-00936,2024-06-29,Saturday,True
3739,(판)지급수수료,66000,0.0,606-71-00375,2024-06-29,Saturday,True
3740,(판)지급수수료,12000,0.0,635-51-00865,2024-06-29,Saturday,True


In [223]:
# 공휴일 여부
import holidays
kr_holidays = holidays.KR(years=year)
df2['공휴일여부'] = df2['날자'].isin(kr_holidays)

<ipython-input-223-d3d0b8e02d39>:4: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df2['공휴일여부'] = df2['날자'].isin(kr_holidays)
<ipython-input-223-d3d0b8e02d39>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['공휴일여부'] = df2['날자'].isin(kr_holidays)


In [224]:
df2.head()

,차변계정,공급가액,세액,사업자등록번호,날자,요일,주말여부,공휴일여부
0,(판)지급수수료,37273,3727.0,120-81-33347,2024-01-09,Tuesday,False,False
1,(판)소모품비,321819,32181.0,513-86-01926,2024-01-16,Tuesday,False,False
2,(판)여비교통비,37276,3724.0,627-37-00247,2024-01-16,Tuesday,False,False
3,(판)소모품비,150000,15000.0,726-85-00088,2024-01-16,Tuesday,False,False
4,(판)여비교통비,250000,25000.0,739-11-00628,2024-01-17,Wednesday,False,False


In [225]:
# 날자 제거
df2 = df2.drop('날자',axis=1)

In [226]:
# 범주형 데이터 인코딩
# 범주형, 사업자 등록번호 : 범주형 -> 라벨인코딩, 원-핫 인코딩
# 요일, 주말여부,공휴일 여부 : 원-핫 인코딩
# 공급가액, 세액 : 수치형... 표준화 또는 정규화

In [227]:
# 범주형 데이터의 개수가  10개 이하인 항목은 제거
temp = df2['차변계정'].value_counts()
remove_values = temp[temp<=10].index
df2 = df2[~df2['차변계정'].isin(remove_values)]

In [228]:
df2['차변계정'].value_counts()

,count
차변계정,
(판)소모품비,1072
(판)여비교통비,1023
(판)복리후생비,689
(판)지급수수료,286
상품,180
미지급금,157
(판)접대비(기업업무추진비),109
(판)통신비,84
(판)차량유지비,58


In [229]:
# 타겟분리
y = df2['차변계정']
X = df2.drop(columns=['차변계정'])

In [230]:
numerical_features = X.describe().columns
numerical_features

Index(['공급가액', '세액'], dtype='object')

In [231]:
categorical_features = X.describe(include=['object','bool']).columns
categorical_features

Index(['사업자등록번호', '요일', '주말여부', '공휴일여부'], dtype='object')

In [232]:
# 수치형 데이터는 표준화
# 범주형 데이터는 원-핫인코딩
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# 전처리 파이프라인 생성
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [233]:
# 데이터 전처리 및 모데 학습을 위한 파이프라인
pipeline = Pipeline(steps=[('preprocessor',preprocessor)])

In [234]:
X.shape

(3709, 6)

In [235]:
# 학습데이터 전처리
X_processed = pipeline.fit_transform(X) # 규칙을 적용하고 변환

In [236]:
X_processed.shape

(3709, 912)

In [237]:
# 클래스 불균형 문제 발생
# oversample
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
smote = SMOTE(sampling_strategy='auto',random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_processed, y)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [238]:
y_resampled.value_counts()

,count
차변계정,
(판)지급수수료,1072
(판)소모품비,1072
(판)여비교통비,1072
(판)접대비(기업업무추진비),1072
(판)통신비,1072
(판)차량유지비,1072
상품,1072
(판)복리후생비,1072
미지급금,1072


In [240]:
X_resampled.shape, y_resampled.shape

((11792, 912), (11792,))

In [244]:
# 데이터 분리
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =  train_test_split(X_resampled,y_resampled, stratify=y_resampled , test_size=0.2,  random_state=42)

In [245]:
len(y_train.unique()), len(y_test.unique())

(11, 11)

In [246]:
X_train.shape, X_test.shape

((9433, 912), (2359, 912))

In [247]:
# 모델 학습
from sklearn.ensemble import RandomForestClassifier
# 모델 평가
from sklearn.metrics import classification_report

In [248]:
# 모델 생성
model = RandomForestClassifier(random_state=42)
# 모델 학습
model.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

In [249]:
# 예측
y_pred = model.predict(X_test)

In [251]:
y_pred[1], y_test.to_numpy()[1]

('(판)복리후생비', '(판)복리후생비')

In [255]:
y_pred,y_test

(array(['(판)차량유지비', '(판)복리후생비', '(판)차량유지비', ..., '(판)의료소모품', '(판)지급수수료',
        '(판)통신비'], dtype=object),
 7585    (판)차량유지비
 3097    (판)복리후생비
 7766    (판)차량유지비
 2729    (판)복리후생비
 4201    (판)의료소모품
           ...   
 1456     (판)소모품비
 6462    (판)지급수수료
 4726    (판)의료소모품
 6878    (판)지급수수료
 8648      (판)통신비
 Name: 차변계정, Length: 2359, dtype: object)

In [256]:
# 정확도 accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.910555320050869

In [258]:
print(classification_report(y_test,y_pred))

                 precision    recall  f1-score   support

       (판)복리후생비       0.83      0.88      0.85       214
        (판)소모품비       0.70      0.67      0.69       214
       (판)여비교통비       0.77      0.79      0.78       214
       (판)의료소모품       1.00      1.00      1.00       215
(판)접대비(기업업무추진비)       1.00      0.98      0.99       215
       (판)지급수수료       0.90      0.88      0.89       214
       (판)차량유지비       0.99      1.00      1.00       215
         (판)통신비       1.00      1.00      1.00       215
           미지급금       0.97      0.99      0.98       215
            미추천       1.00      1.00      1.00       214
             상품       0.85      0.83      0.84       214

       accuracy                           0.91      2359
      macro avg       0.91      0.91      0.91      2359
   weighted avg       0.91      0.91      0.91      2359



In [ ]:
# precision    : 정밀도  모델이 예측한 정답 중에서  실제로 정답인 비율
# recall  : 재현율   실제 정답중에서 모델이 올바르게 예측한 비율
# f1_score : precision 와 recall의 조화 평균